In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from tqdm import tqdm
from matplotlib import pyplot as plt
import random
import cProfile, pstats, io, string
import distance
import math

# Question 2 - Clustering

In [3]:
dt = pd.read_csv('Reviews.csv', sep=",", usecols = [i for i in range(1,10)])

In [4]:
dt.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
dt.describe()

,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,1.743817,2.22881,4.183199,1.296257e+09
std,7.636513,8.28974,1.310436,4.804331e+07
min,0.000000,0.00000,1.000000,9.393408e+08
25%,0.000000,0.00000,4.000000,1.271290e+09
50%,0.000000,1.00000,5.000000,1.311120e+09
75%,2.000000,2.00000,5.000000,1.332720e+09
max,866.000000,923.00000,5.000000,1.351210e+09


## Clean up data

In [6]:
dt = dt.drop_duplicates(subset=["UserId","ProfileName","Time","Text"], keep='first', inplace=False)
dt = dt[dt["HelpfulnessNumerator"] <= dt["HelpfulnessDenominator"]] # remove the numerator greater than denominator

In [7]:
dt.describe()

,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,393931.000000,393931.000000,393931.000000,3.939310e+05
mean,1.734393,2.204805,4.179397,1.296449e+09
std,6.872280,7.534791,1.311927,4.840344e+07
min,0.000000,0.000000,1.000000,9.393408e+08
25%,0.000000,0.000000,4.000000,1.271376e+09
50%,0.000000,1.000000,5.000000,1.311638e+09
75%,2.000000,2.000000,5.000000,1.332979e+09
max,866.000000,923.000000,5.000000,1.351210e+09


In [8]:
dt.isnull().sum()

ProductId                  0
UserId                     0
ProfileName               11
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                    3
Text                       0
dtype: int64

There is a very few number of null values. Let's fill them:

In [9]:
dt['ProfileName'].fillna('', inplace=True)
dt['Summary'].fillna('', inplace=True)

In [10]:
dt.isnull().sum()

ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

## Tokenize the text 

In [11]:
product_id = []
summary = []
reviews = []
new_df = pd.DataFrame()
for product, group in dt.groupby('ProductId'):
    product_id.append(product)
    reviews.append(" ".join(list(group['Text'])))
    summary.append(" ".join(list(map(str,group['Summary']))))
    
new_df['ProductId'] = product_id
new_df['reviews'] = reviews

In [12]:
def clean_text(text):
    stop_words = set(stopwords.words('english')) # obtain the stop words
    good_words = [] # save the correct words to consider like tokens
    tokenizer = RegexpTokenizer("[\w']+") # function to recognize the tokens
    words = tokenizer.tokenize(text) # tokenize the text 
    for word in words:
        # check if the word is lower and it isn't a stop word or a number
        if word.lower() not in stop_words and word.isalpha(): 
            word = PorterStemmer().stem(word) # use the stemmer function
            good_words.append(word.lower()) # insert the good token to lower case
        
    return good_words

In [13]:
%%time
# use the interest column..
new_df["reviews"] = [clean_text(x) for x in list(new_df["reviews"])]
new_df.head()

Wall time: 5min 27s


,ProductId,reviews
0,0006641040,"[day, person, say, chicken, soup, probabl, go,..."
1,141278509X,"[product, archer, farm, best, drink, mix, ever..."
2,2734888454,"[dog, love, chicken, product, china, wont, buy..."
3,2841233731,"[book, easi, read, ingredi, avail, store, unli..."
4,7800648702,"[came, huge, tin, much, bigger, expect, cooki,..."


In [14]:
%%time

tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, min_df = 0.01)
tfidf_matrix = tfidf.fit_transform(new_df["reviews"])

Wall time: 5.07 s


In [15]:
%%time
len(tfidf.vocabulary_)

Wall time: 0 ns


1740

In [16]:
%%time
dn = pd.DataFrame.sparse.from_spmatrix(tfidf_matrix)

Wall time: 98.7 ms


In [17]:
dn

,0,1,2,3,4,5,6,7,8,9,...,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739
0,0.013209,0.006562,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.010587,...,0.0,0.007034,0.0,0.010472,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.191621,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67619,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
67620,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
67621,0.000000,0.000000,0.328941,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
67622,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [18]:
svd = TruncatedSVD(n_components=350)
new_dn = svd.fit_transform(dn)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum()*100)

[0.01284971 0.02532599 0.01783335 0.01390644 0.01191672 0.01048781
 0.00890187 0.00793407 0.00694107 0.00641578 0.00598934 0.00566482
 0.0052877  0.00500848 0.00488534 0.00468479 0.00457318 0.00438951
 0.00430708 0.00413448 0.0040118  0.00400596 0.00390218 0.00386503
 0.00378525 0.0037135  0.00361557 0.00355142 0.00348168 0.00345191
 0.00333923 0.00323925 0.00320581 0.0031668  0.00310885 0.00302393
 0.00297262 0.00289839 0.00287718 0.00279555 0.00277909 0.0027433
 0.00268713 0.00267405 0.00263782 0.00262516 0.0026132  0.00256031
 0.00252261 0.0024905  0.00246319 0.00245834 0.00241211 0.00236393
 0.0023514  0.00230668 0.0022922  0.00225895 0.00222967 0.00221729
 0.00220726 0.00218499 0.00215404 0.00213932 0.00212021 0.00210534
 0.00208764 0.00207045 0.00204699 0.00202513 0.00199147 0.0019846
 0.00196503 0.00194596 0.00193025 0.00191566 0.00190435 0.00188604
 0.00187567 0.00187468 0.00185448 0.00184012 0.00181172 0.00180688
 0.00179392 0.00177586 0.00176108 0.00174449 0.0017351  0.001727

In [19]:
def profile(fnc):

    """a decorator that uses Cprofile to profile a function"""

    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc (*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

## K-Means 1


In [20]:
from collections import defaultdict


def calc_distance(X1, X2):
    return np.sqrt(sum((X1 - X2)**2))


def initialization(dn, k):
    temp = random.sample(list(dn), k)
    return np.array(temp)


def assign_cluster(new_dn, centroids):
    print('assign_cluster')
    clusters = defaultdict(list)
    clusters_idx = defaultdict(list)
    for i, x in enumerate(new_dn):
        centroid = np.argmin([np.array([np.linalg.norm(x-mu_j) for j,mu_j in enumerate(centroids)])])
        clusters[centroid].append(x)
        clusters_idx[centroid].append(i)
    return clusters, clusters_idx


@profile
def kmeans(new_dn, k, centroids = None, cluster_prec = defaultdict(list),init=True):
    if init:
        centroids = initialization(new_dn, k)

    clusters, clusters_idx = assign_cluster(new_dn, centroids)
    if clusters_idx == cluster_prec[1]:
        print('stop condition')
        return clusters
    else:
        centroids = update_centroid(centroids, clusters, k)
        return kmeans(new_dn, k, centroids, cluster_prec = (clusters.copy(), clusters_idx),init=False)

def update_centroid(centroids,clusters,k):
    upd_c = np.array([np.round(np.mean(clusters[i], axis = 0),5) if i in clusters.keys() else centroids[i] for i in range(k)])
    return upd_c

## K Means 2

In [63]:
class Kmeans():

    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.prev_labels = [1]
        self.labels = []


    def initialize_algo(self, matrix):
        random_indices = np.random.choice(len(matrix), size= self.n_clusters, replace=False)
        self.centroids = matrix[random_indices, :]



    def stop_iteration_flag(self):
        if self.labels == self.prev_labels:
            return True
        else:
            return False

    
    def compute_distance(self, vec1, vec2):
        return np.linalg.norm(vec1 - vec2)

    
    def assign_clusters(self, matrix):
        self.clusters = {}
        self.prev_labels = self.labels.copy()
        self.labels = []
        
        for row in matrix:
            centroid_idx = np.argmin([self.compute_distance(row, centroid) for centroid in self.centroids])
            self.clusters.setdefault(centroid_idx, []).append(row)
            self.labels.append(centroid_idx)

    
    def update_centroids(self):
        self.centroids = [np.mean(i, axis = 0) for i in self.clusters.values()]

    @profile
    def fit(self, matrix):
        self.initialize_algo(matrix)
        iter_count = 0
        while not self.stop_iteration_flag():
            print("iteration no. {0}".format(iter_count))
            self.assign_clusters(matrix)
            self.update_centroids()
            iter_count += 1
        
        return self.labels

## Test K Means

In [64]:
model_test = Kmeans(5)
ans = model_test.fit(new_dn)

iteration no. 0
iteration no. 1
iteration no. 2
iteration no. 3
iteration no. 4
iteration no. 5
iteration no. 6
iteration no. 7
iteration no. 8
iteration no. 9
iteration no. 10
iteration no. 11
iteration no. 12
iteration no. 13
iteration no. 14
iteration no. 15
iteration no. 16
iteration no. 17
iteration no. 18
iteration no. 19
iteration no. 20
iteration no. 21
iteration no. 22
iteration no. 23
iteration no. 24
iteration no. 25
iteration no. 26
iteration no. 27
iteration no. 28
iteration no. 29
iteration no. 30
iteration no. 31
iteration no. 32
iteration no. 33
iteration no. 34
iteration no. 35
iteration no. 36
iteration no. 37
iteration no. 38
iteration no. 39
iteration no. 40
iteration no. 41
iteration no. 42
iteration no. 43
iteration no. 44
iteration no. 45
iteration no. 46
iteration no. 47
iteration no. 48
iteration no. 49
iteration no. 50
iteration no. 51
iteration no. 52
iteration no. 53
iteration no. 54
iteration no. 55
iteration no. 56
iteration no. 57
iteration no. 58
iterati

In [ ]:
my_dict = {}
for idx, key in enumerate(ans):
    my_dict.setdefault(key, set()).add(idx)
    

In [ ]:
%%time

type(initialization(new_dn,5))

In [ ]:
%%time
c = kmeans(new_dn, 5)
#update_centroid(initialization(new_dn, 5), c, 5)

In [ ]:
c.keys()

In [46]:
model = KMeans(init = 'k-means++', n_clusters = 5)

In [47]:
slice = new_dn[:] # only column 3 to 6

In [48]:
model.fit(slice)

KMeans(n_clusters=5)

In [49]:
cluster = model.predict(slice)

In [50]:
cluster[:10]

array([1, 1, 2, 1, 1, 2, 1, 1, 1, 1])

In [ ]:
new_dataframe = pd.DataFrame()
new_dataframe['ProductId'] = new_df['ProductId']
new_dataframe['Cluster'] = cluster

In [ ]:
dt.loc[(dt['ProductId'] == 'B009UOFU20') | (dt['ProductId'] == 'B009UOFTUI')]

In [ ]:
new_dataframe.groupby('Cluster').count().sort_values(by='ProductId')
lista = new_dataframe.loc[new_dataframe['Cluster'] == 52]
lista2 = lista['ProductId']
new_df.loc[new_df['ProductId'].isin(lista2)]

In [ ]:
slice['cluster'] = cluster

Let's plot the results

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig = plt.figure()
for cluster in range(0, 10):
    plt.scatter(slice[slice.cluster == cluster].Score, slice[slice.cluster == cluster].HelpfulnessNumerator)
plt.show()

In [ ]:
model = KMeans(verbose=False)
slice = dataset.iloc[:,3:6]
model.fit(slice)
cluster_values = model.predict(slice)
dataset['cluster'] = cluster_values
fig = plt.figure()
for cluster in range(0,10):
    plt.scatter(dataset[dataset.cluster == cluster].Score, dataset[dataset.cluster == cluster].HelpfulnessDenominator)
plt.show()

In [ ]:
fig = plt.figure()
for cluster in range(0,10):
    plt.scatter(dataset[dataset.cluster == cluster].HelpfulnessNumerator, dataset[dataset.cluster == cluster].HelpfulnessDenominator)
plt.show()

In [ ]:
from tqdm import tqdm

In [ ]:
slice = dataset.iloc[:,3:6]

In [ ]:
elbow = {}
for k in tqdm(range(3,20)):
    elbow_model = KMeans(n_clusters=k)
    elbow_model.fit_predict(slice)
    elbow[k] = elbow_model.inertia_ 

In [ ]:
plt.plot(list(elbow.keys()), list(elbow.values()))

In [ ]:
products_per_cluster = {}
for cluster in range(0,6):
    products_per_cluster[cluster] = len(dataset[dataset.cluster == cluster].ProductId.unique())

In [ ]:
products_per_cluster

In [ ]:
users_per_cluster = {}
for cluster in range(0,6):
    users_per_cluster[cluster] = len(dataset[dataset.cluster == cluster].UserId.unique())

In [ ]:
users_per_cluster

In [ ]:
row_per_cluster = {}
for cluster in range(0,6):
    row_per_cluster[cluster] = len(dataset[dataset.cluster == cluster])

In [ ]:
row_per_cluster